# Tekstklassifiseringsoppgave

I denne modulen skal vi starte med en enkel tekstklassifiseringsoppgave basert på **[AG_NEWS](http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html)**-datasettet: vi skal klassifisere nyhetsoverskrifter i en av 4 kategorier: Verden, Sport, Næringsliv og Vitenskap/Teknologi.

## Datasettet

For å laste inn datasettet, skal vi bruke **[TensorFlow Datasets](https://www.tensorflow.org/datasets)**-APIet.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

# In this tutorial, we will be training a lot of models. In order to use GPU memory cautiously,
# we will set tensorflow option to grow GPU memory allocation when required.
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

dataset = tfds.load('ag_news_subset')

Vi kan nå få tilgang til trenings- og testdelene av datasettet ved å bruke `dataset['train']` og `dataset['test']` henholdsvis:


In [3]:
ds_train = dataset['train']
ds_test = dataset['test']

print(f"Length of train dataset = {len(ds_train)}")
print(f"Length of test dataset = {len(ds_test)}")

Length of train dataset = 120000
Length of test dataset = 7600


La oss skrive ut de første 10 nye overskriftene fra datasettet vårt:


In [4]:
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

for i,x in zip(range(5),ds_train):
    print(f"{x['label']} ({classes[x['label']]}) -> {x['title']} {x['description']}")

3 (Sci/Tech) -> b'AMD Debuts Dual-Core Opteron Processor' b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.'
1 (Sports) -> b"Wood's Suspension Upheld (Reuters)" b'Reuters - Major League Baseball\\Monday announced a decision on the appeal filed by Chicago Cubs\\pitcher Kerry Wood regarding a suspension stemming from an\\incident earlier this season.'
2 (Business) -> b'Bush reform may have blue states seeing red' b'President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local taxes may be in administration planners #39; sights, news reports say.'
3 (Sci/Tech) -> b"'Halt science decline in schools'" b'Britain will run out of leading scientists unless science education is improved, says Professor Colin Pillinger.'
1 (Sports) -> b'Gerrard leaves practice' b'London, England (Sports Network

## Tekstvektorisering

Nå må vi konvertere tekst til **tall** som kan representeres som tensorer. Hvis vi ønsker representasjon på ordnivå, må vi gjøre to ting:

* Bruke en **tokenizer** for å dele opp teksten i **tokens**.
* Bygge et **vokabular** av disse tokens.

### Begrense vokabularstørrelsen

I eksempelet med AG News-datasettet er vokabularstørrelsen ganske stor, mer enn 100 000 ord. Generelt sett trenger vi ikke ord som sjelden forekommer i teksten — bare noen få setninger vil inneholde dem, og modellen vil ikke lære noe av dem. Derfor gir det mening å begrense vokabularstørrelsen til et mindre antall ved å sende et argument til vektorisatorens konstruktør:

Begge disse trinnene kan håndteres ved hjelp av **TextVectorization**-laget. La oss opprette vektorisatorobjektet og deretter kalle `adapt`-metoden for å gå gjennom all tekst og bygge et vokabular:


In [5]:
vocab_size = 50000
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size)
vectorizer.adapt(ds_train.take(500).map(lambda x: x['title']+' '+x['description']))

> **Merk** at vi kun bruker en delmengde av hele datasettet for å bygge et ordforråd. Vi gjør dette for å redusere kjøretiden og unngå at du må vente. Imidlertid tar vi risikoen for at noen av ordene fra hele datasettet ikke blir inkludert i ordforrådet og dermed blir ignorert under trening. Derfor bør bruk av hele ordforrådet og gjennomgang av hele datasettet under `adapt` øke den endelige nøyaktigheten, men ikke vesentlig.

Nå kan vi få tilgang til det faktiske ordforrådet:


In [6]:
vocab = vectorizer.get_vocabulary()
vocab_size = len(vocab)
print(vocab[:10])
print(f"Length of vocabulary: {vocab_size}")

['', '[UNK]', 'the', 'to', 'a', 'in', 'of', 'and', 'on', 'for']
Length of vocabulary: 5335


Ved å bruke vektorisatoren kan vi enkelt kode enhver tekst til et sett med tall:


In [7]:
vectorizer('I love to play with my words')

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([ 112, 3695,    3,  304,   11, 1041,    1], dtype=int64)>

## Bag-of-words tekstrepresentasjon

Fordi ord representerer mening, kan vi noen ganger forstå betydningen av en tekst ved bare å se på de individuelle ordene, uavhengig av rekkefølgen i setningen. For eksempel, når vi klassifiserer nyheter, vil ord som *vær* og *snø* sannsynligvis indikere *værmelding*, mens ord som *aksjer* og *dollar* vil peke mot *finansnyheter*.

**Bag-of-words** (BoW) vektorrepresentasjon er den enkleste tradisjonelle vektorrepresentasjonen å forstå. Hvert ord er knyttet til en vektorindeks, og et vektorelement inneholder antall forekomster av hvert ord i et gitt dokument.

![Bilde som viser hvordan en bag-of-words vektorrepresentasjon lagres i minnet.](../../../../../translated_images/no/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Note**: Du kan også tenke på BoW som en sum av alle én-hot-kodede vektorer for individuelle ord i teksten.

Nedenfor er et eksempel på hvordan man kan generere en bag-of-words representasjon ved hjelp av Scikit Learn python-biblioteket:


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
sc_vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
sc_vectorizer.fit_transform(corpus)
sc_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

Vi kan også bruke Keras-vektoriseringen som vi definerte ovenfor, konvertere hvert ordnummer til en one-hot-koding og legge sammen alle disse vektorene:


In [9]:
def to_bow(text):
    return tf.reduce_sum(tf.one_hot(vectorizer(text),vocab_size),axis=0)

to_bow('My dog likes hot dogs on a hot day.').numpy()

array([0., 5., 0., ..., 0., 0., 0.], dtype=float32)

> **Merk**: Du kan bli overrasket over at resultatet skiller seg fra det forrige eksempelet. Årsaken er at i Keras-eksempelet tilsvarer lengden på vektoren vokabularstørrelsen, som ble bygget fra hele AG News-datasettet, mens vi i Scikit Learn-eksempelet bygde vokabularet fra eksempelteksten underveis.


## Trene BoW-klassifikatoren

Nå som vi har lært hvordan vi bygger bag-of-words-representasjonen av teksten vår, la oss trene en klassifikator som bruker den. Først må vi konvertere datasettet vårt til en bag-of-words-representasjon. Dette kan gjøres ved å bruke `map`-funksjonen på følgende måte:


In [11]:
batch_size = 128

ds_train_bow = ds_train.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)
ds_test_bow = ds_test.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)

La oss nå definere et enkelt klassifiserings-nevralt nettverk som inneholder ett lineært lag. Inngangsstørrelsen er `vocab_size`, og utgangsstørrelsen tilsvarer antall klasser (4). Fordi vi løser en klassifiseringsoppgave, er den endelige aktiveringsfunksjonen **softmax**:


In [12]:
model = keras.models.Sequential([
    keras.layers.Dense(4,activation='softmax',input_shape=(vocab_size,))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train_bow,validation_data=ds_test_bow)

938/938 [==============================] - 66s 70ms/step - loss: 0.6144 - acc: 0.8427 - val_loss: 0.4416 - val_acc: 0.8697


Siden vi har 4 klasser, er en nøyaktighet på over 80% et godt resultat.

## Trene en klassifiserer som ett nettverk

Siden vektoriseringen også er et Keras-lag, kan vi definere et nettverk som inkluderer det, og trene det fra start til slutt. På denne måten trenger vi ikke å vektorisere datasettet ved hjelp av `map`, vi kan bare sende det originale datasettet til inngangen av nettverket.

> **Note**: Vi må fortsatt bruke `map` på datasettet vårt for å konvertere felt fra ordbøker (som `title`, `description` og `label`) til tupler. Men når vi laster inn data fra disk, kan vi bygge et datasett med den nødvendige strukturen fra starten av.


In [13]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

inp = keras.Input(shape=(1,),dtype=tf.string)
x = vectorizer(inp)
x = tf.reduce_sum(tf.one_hot(x,vocab_size),axis=1)
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
model.summary()

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 5335)        0         
                                                                 
 tf.math.reduce_sum (TFOpLam  (None, 5335)             0         
 bda)                                                            
                                                                 
 dense_2 (Dense)             (None, 4)                 21344     
                                                                 
Total params: 21,344
Trainable params: 21,344
Non-trainable p

## Bigrammer, trigrammer og n-grammer

En begrensning med bag-of-words-tilnærmingen er at noen ord er en del av flerordsuttrykk. For eksempel har ordet 'hot dog' en helt annen betydning enn ordene 'hot' og 'dog' i andre sammenhenger. Hvis vi alltid representerer ordene 'hot' og 'dog' med de samme vektorene, kan det forvirre modellen vår.

For å løse dette brukes ofte **n-gram-representasjoner** i metoder for dokumentklassifisering, der frekvensen av hvert ord, to-ordsuttrykk eller tre-ordsuttrykk er en nyttig funksjon for å trene klassifikatorer. I bigram-representasjoner, for eksempel, legger vi til alle ordpar i vokabularet, i tillegg til de opprinnelige ordene.

Nedenfor er et eksempel på hvordan man kan generere en bigram bag-of-words-representasjon ved hjelp av Scikit Learn:


In [14]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Den største ulempen med n-gram-tilnærmingen er at vokabularstørrelsen begynner å vokse ekstremt raskt. I praksis må vi kombinere n-gram-representasjonen med en teknikk for dimensjonsreduksjon, slik som *embeddings*, som vi skal diskutere i neste enhet.

For å bruke en n-gram-representasjon i vårt **AG News**-datasett, må vi sende `ngrams`-parameteren til vår `TextVectorization`-konstruktør. Lengden på et bigram-vokabular er **betydelig større**, i vårt tilfelle er det mer enn 1,3 millioner tokens! Derfor gir det mening å begrense bigram-tokens til et rimelig antall.

Vi kunne brukt den samme koden som ovenfor for å trene klassifisereren, men det ville vært svært minne-ineffektivt. I neste enhet skal vi trene bigram-klassifisereren ved hjelp av embeddings. I mellomtiden kan du eksperimentere med bigram-klassifiseringstrening i denne notatboken og se om du kan oppnå høyere nøyaktighet.


## Automatisk beregning av BoW-vektorer

I eksempelet ovenfor beregnet vi BoW-vektorer manuelt ved å summere one-hot-enkodingene av individuelle ord. Men den nyeste versjonen av TensorFlow lar oss beregne BoW-vektorer automatisk ved å sende parameteren `output_mode='count'` til vektorisatorens konstruktør. Dette gjør det betydelig enklere å definere og trene modellen vår:


In [15]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='count'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 7s 7ms/step - loss: 0.5929 - acc: 0.8486 - val_loss: 0.4168 - val_acc: 0.8772


## Termfrekvens - invers dokumentfrekvens (TF-IDF)

I BoW-representasjon vektes ordforekomster ved hjelp av samme teknikk uavhengig av selve ordet. Det er imidlertid tydelig at hyppige ord som *en* og *i* er langt mindre viktige for klassifisering enn spesialiserte termer. I de fleste NLP-oppgaver er noen ord mer relevante enn andre.

**TF-IDF** står for **termfrekvens - invers dokumentfrekvens**. Det er en variant av bag-of-words, der man i stedet for en binær 0/1-verdi som indikerer tilstedeværelsen av et ord i et dokument, bruker en flyttallsverdi som er relatert til hvor ofte ordet forekommer i korpuset.

Mer formelt er vekten $w_{ij}$ til et ord $i$ i dokumentet $j$ definert som:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
hvor
* $tf_{ij}$ er antall forekomster av $i$ i $j$, altså BoW-verdien vi har sett tidligere
* $N$ er antall dokumenter i samlingen
* $df_i$ er antall dokumenter som inneholder ordet $i$ i hele samlingen

TF-IDF-verdien $w_{ij}$ øker proporsjonalt med antall ganger et ord vises i et dokument og justeres ned basert på antall dokumenter i korpuset som inneholder ordet. Dette bidrar til å kompensere for det faktum at noen ord forekommer oftere enn andre. For eksempel, hvis ordet vises i *alle* dokumentene i samlingen, er $df_i=N$, og $w_{ij}=0$, og disse termene vil bli fullstendig ignorert.

Du kan enkelt lage TF-IDF-vektorisering av tekst ved hjelp av Scikit Learn:


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

I Keras kan `TextVectorization`-laget automatisk beregne TF-IDF-frekvenser ved å sende parameteren `output_mode='tf-idf'`. La oss gjenta koden vi brukte ovenfor for å se om bruk av TF-IDF øker nøyaktigheten:


In [17]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='tf-idf'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 12s 12ms/step - loss: 0.4197 - acc: 0.8662 - val_loss: 0.3432 - val_acc: 0.8849


## Konklusjon

Selv om TF-IDF-representasjoner gir frekvensvekter til ulike ord, er de ikke i stand til å representere mening eller rekkefølge. Som den kjente lingvisten J. R. Firth sa i 1935: "Den fullstendige betydningen av et ord er alltid kontekstuell, og ingen studie av betydning utenfor kontekst kan tas seriøst." Senere i kurset vil vi lære hvordan vi kan fange opp kontekstuell informasjon fra tekst ved hjelp av språklig modellering.



---

**Ansvarsfraskrivelse**:  
Dette dokumentet er oversatt ved hjelp av AI-oversettelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selv om vi streber etter nøyaktighet, vær oppmerksom på at automatiske oversettelser kan inneholde feil eller unøyaktigheter. Det originale dokumentet på sitt opprinnelige språk bør anses som den autoritative kilden. For kritisk informasjon anbefales profesjonell menneskelig oversettelse. Vi er ikke ansvarlige for misforståelser eller feiltolkninger som oppstår ved bruk av denne oversettelsen.
